In [ ]:
!pip install -q \
  transformers \
  peft \
  accelerate \
  bitsandbytes \
  trl \
  datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.5/532.5 kB 41.0 MB/s eta 0:00:00


In [ ]:
!pip uninstall -y transformers
!pip install -U transformers sentencepiece accelerate


Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 58.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/Teaching_clone/train_dataset.jsonl",
    split="train"
)

print(dataset[0])


Generating train split: 0 examples [00:00, ? examples/s]

{'data': [{'instruction': 'Explain gradient descent in deep learning in simple Hinglish, step by step, using intuition and examples.', 'output': 'Dekho, pehle ek problem samajhte hain. Jab hum neural network ko train karte hain, toh humare paas ek loss function hota hai jo batata hai ki model kitna galat hai. Ab sawal yeh hai ki hum is loss ko minimize kaise karein? Agar hum randomly weights change karein, toh bahut time lagega aur ho sakta hai kabhi minimize ho hi na. Is problem ko solve karne ke liye gradient descent use hota hai. Socho ek pahaad pe tum khade ho aur tumhe neeche jaana hai. Sabse fast way kya hoga? Jo direction sabse steep downward ho, wahi chaloge na? Gradient descent bhi yahi karta hai. Loss function ki slope dekh ke, weights ko us direction mein move karta hai jahan loss kam ho. Learning rate decide karta hai ki kitne bade steps loge. Agar bahut bada step loge toh overshoot ho jaoge, agar bahut chhota loge toh bahut time lagega.'}, {'instruction': 'Explain backprop

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    trust_remote_code=True,
    use_fast = False
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 3,686,400 || all params: 3,089,625,088 || trainable%: 0.1193


In [ ]:
def format_example(example):
    messages = [
        {"role": "system", "content": "You are an experienced Indian teacher who explains concepts in simple Hinglish."},
        {"role": "user", "content": example["instruction"]},
        {"role": "assistant", "content": example["output"]},
    ]
    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Teaching_clone/nitish_style_lora_1",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=15,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none",
    optim="adamw_torch"
)


In [ ]:
from trl import SFTTrainer
from datasets import Dataset # Import Dataset from the datasets library

# Extract the list of examples from the dataset and create a new Dataset object
# Assuming dataset is a Dataset object with a single row, where that row is {'data': [list_of_actual_samples]}
# and that list_of_actual_samples is a list of dictionaries like {'instruction': '...', 'output': '...'}
processed_train_dataset = Dataset.from_list(dataset[0]['data'])

trainer = SFTTrainer(
    model=model,
    train_dataset=processed_train_dataset,
    formatting_func=format_example,
    args=training_args
)


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:2111: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(


Applying formatting function to train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,2.832600
20,2.400000
30,2.110900
40,1.988100
50,1.889400
60,1.797500
70,1.767800
80,1.692900
90,1.635100
100,1.567700


TrainOutput(global_step=195, training_loss=1.7040426694429838, metrics={'train_runtime': 557.7321, 'train_samples_per_second': 2.689, 'train_steps_per_second': 0.35, 'total_flos': 7744151942553600.0, 'train_loss': 1.7040426694429838})

In [19]:
trainer.model.save_pretrained("/content/drive/MyDrive/nitish_style_lora_final")
tokenizer.save_pretrained("/content/drive/MyDrive/nitish_style_lora_final")


('/content/drive/MyDrive/nitish_style_lora_final/tokenizer_config.json',
 '/content/drive/MyDrive/nitish_style_lora_final/special_tokens_map.json',
 '/content/drive/MyDrive/nitish_style_lora_final/chat_template.jinja',
 '/content/drive/MyDrive/nitish_style_lora_final/vocab.json',
 '/content/drive/MyDrive/nitish_style_lora_final/merges.txt',
 '/content/drive/MyDrive/nitish_style_lora_final/added_tokens.json')

In [ ]:
def ask(prompt):
    messages = [
        {"role": "system", "content": "You are an experienced Indian teacher who explains concepts in simple Hinglish."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.5,
        top_p=0.9,
        do_sample=True
    )
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [20]:
ask("Explain overfitting in simple Hinglish.")

system
You are an experienced Indian teacher who explains concepts in simple Hinglish.
user
Explain overfitting in simple Hinglish.
assistant
Pehle ek example samajhte hain. Suppose yeh bhi tha training data mein train kiya, aur test kiye. Isse 90% accuracy dekh raha hai. Aur pehle time test kiya matlab is model ko validation dataset pe bahut well predict karne do. Par yahan pahunchte hain overfitting ka problem. Jaise 100% accuracy ho jaaye matlab. Iska kya meaning hai? Is situation ke liye model ko bahut specific data ko learn karne se problem hota hai. Training dataset ke features ko bahut well capture karne padte hain, par new data pe bahut poor results aate hain. Yeh problem ek extreme case hai jo training dataset ke data ko bahut well capture karne ke liye chahiye. Socho ek student hai jo textbook ke questions solve karne ke liye baaki content ignore karta hai. Jab question textbook mein nahi hota toh wo wrong answers deta hai. Sabse pheli solution yaar kya hai? Regularization us